# C) Poisson problem

- **<font color='green'>[RUN & OBSERVE]</font>** $\rightarrow$ the cell should be run directly without modification
- **<font color='orange'>[RUN & PLAY]</font>** $\rightarrow$ the cell can be run directly, but some parameters should be changed interactively
- **<font color='red'>[FILL & RUN]</font>**    $\rightarrow$ the cell should be filled before being run
- **<font color='magenta'>[FILL & PLAY]</font>** $\rightarrow$ the cell should be filled, and then some parameters should be changed interactively.

We consider the following classical Poisson problem (inspired from electrostatics)

$$ 
\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\text{some boundary conditions} &  & & \text{on} & \partial \Omega
\end{array}
$$

We consider a unit square domain, with identified boundaries.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from utils.myGeometries import square
from ngsolve.webgui import Draw

Omega = square(maxh=0.1)    # generates the domain
print(f"boundary labels = {Omega.GetBoundaries()}")
Draw(Omega)

_____
## 1) Variational formulation

**Method** : 
1. Chose the appropriate function space
2. Multiply by a test function $v$ and integrate
3. Decrease the derivative order using Green formula to avoid derivative of discontinuous functions

**NB** : derivate discontinuous functions is possible using distribution theory, so what is before makes sense!

Let's consider $v\in H(\Omega)$ (we leave the definition of $H(\Omega)$ for later). Then

$$ - \int_\Omega v \nabla \cdot (\epsilon \nabla u) = \int_\Omega v \rho $$

- Leibniz formula : $\nabla \cdot (M \vec{A}) = M \nabla \cdot \vec{A} + \vec{A} \cdot \nabla M$
- Divergence theorem : $\int_\Omega \nabla \cdot \vec{A} = \int_{\partial \Omega} \vec{A} \cdot \vec{n}$

$$ \Rightarrow \int_\Omega \nabla v \cdot (\epsilon \nabla u) - \int_\Omega \nabla \cdot (v \epsilon \nabla u) =  \int_\Omega v \rho$$
$$ \Rightarrow \boxed{\int_\Omega \nabla v \cdot (\epsilon \nabla u) - \int_{\partial \Omega} v \epsilon \nabla u \cdot n =  \int_\Omega v \rho}$$

There is now one boundary term. On a part of this boundary, we can have "natural" boundary conditions :
- **Neumann** : $\epsilon\nabla u \cdot n = g $
- **Robin** : $\epsilon\nabla u \cdot n = g - r u $
 
Or essential boundary conditions, meaning we have to change the definition of the function space $H$:
- **Dirichlet** : $u = u_D$
- periodicity, anti-periodicity, etc.

The space $H(\Omega)$ should be $H^1(\Omega)$ + essential boundary conditions.

_____
## 2) Homogeneous Dirichlet & Neumann

We consider the following boundary-value problem, with both Dirichlet & Neumann boundary conditions:
$$ 
\left \{\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\epsilon \nabla u \cdot n & =  & 0 & \text{on}~\Gamma_N ~\text{(left \& top)} \\
 u & =  & 0 & \text{on}~\Gamma_D ~\text{(bottom \& right)} \\
\end{array} \right.
$$

- Homogeneous Neumann is the "default" **natural** boundary condition.
- Since Dirichlet is an **essential** boundary condition it should appear in the space; so

$$H(\Omega) = H^1_0(\Omega) =  \{u\in H^1(\Omega), u = 0 ~\text{on}~\Gamma_D \} $$


|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from ngsolve import H1
fes = H1(Omega, order = 1, dirichlet  = "bottom|right")

We can now define the linear and bilinear forms symbolically, using the syntaxes :
- `A = BilinearForm( <symbolic expression> * dx)`
- or `A = BilinearForm(fes)`, and then `A += <symbolic expression> * dx` for long expressions.
- `A += <symbolic expression> * dx("regionLabel")` is equivalent to restric the integral over a specific region.

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
from ngsolve import grad, dx, LinearForm, BilinearForm

rho, eps = 1., 1. # coefficient value (can be changed)
u, v = fes.TnT()  # Trial aNd Test
A = BilinearForm(fes)
A += .................................
l = LinearForm(fes)
l += .................................

Now we can assemble a matrix and right-hand side vector out of the symbolic expressions.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
K = A.Assemble().mat
f = l.Assemble().vec

We then solve the linear system, considering only the free DoFs, where the Dirichlet condition is not applied.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from ngsolve import GridFunction
sol = GridFunction(fes)
sol.vec.data += K.Inverse( freedofs = fes.FreeDofs() ) * f
Draw(sol, settings = {"deformation" :  1})

Let's have a look at the $\vec{E}$ field :

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
E = ...........................
Draw(E, Omega, vectors={"grid_size" : 10, "offset" : 0.5 }) 

___
### Exercise : 

To integrate on a boundary, we use `ds("name")` where `name` is the label of the boundary (without labels, all boundaries are considered).
Write the variational formulation and solve the following problem :

$$ 
\left \{\begin{array}{rcll}
-\nabla \cdot (\epsilon \nabla u) & = & \rho & \text{in}~\Omega \\
\epsilon \nabla u \cdot n & =  & 0 & \text{on}~\Gamma_{N1} ~\text{(left)} \\
\epsilon \nabla u \cdot n & =  & 1 & \text{on}~\Gamma_{N2} ~\text{(right)} \\
 u & =  & 0 & \text{on}~\Gamma_{D} ~\text{(bottom \& top)} \\
\end{array} \right.
$$

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
#------------------------------------------------------------------------------------
# 1) Function space
fes = ...............................
#------------------------------------------------------------------------------------
# 2) Equation
from ngsolve import ds
rho, eps = 1., 1.
u, v = fes.TnT()  # Trial aNd Test
A = BilinearForm(fes)
A += ...........................
l = LinearForm(fes)
l += ...........................
#------------------------------------------------------------------------------------
# 3) Assembly
K = A.Assemble().mat
f = l.Assemble().vec
#------------------------------------------------------------------------------------
# 4) Solving
sol = GridFunction(fes)
sol.vec.data += ............................
#------------------------------------------------------------------------------------
# 4) Display & post-processing
Draw(sol, settings = {"deformation" :  1})